In [16]:
from sko.ACA import ACA_TSP
from math import sin, asin, cos, sqrt, atan2, radians
import regex as re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np

def get_data():
    # Scrapes the data 
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome('./chromedriver', options = options)

    driver.get("https://www.infoplease.com/world/geography/major-cities-latitude-longitude-and-corresponding-time-zones")
    elem = driver.find_element_by_id("A0001770")
    text = elem.text
    driver.close()
    return text

def get_cities(data):
    # Formats the data into a more convenient lookup table
    rows = data.split('\n')
    cities = []
    for row in rows[2:]:
        city = re.findall('[A-Za-z].*, [A-Za-z]*', row)
        latitude, longitude = re.findall('[0-9]{1,} [0-9]{1,} [SNWE]', row)
        cities.append({'city': city, 
                       'latitude': latitude, 
                       'longitude': longitude})
    return cities

def to_degrees(city):
    dms_to_deg = lambda deg, minutes, direction: (float(deg) + float(minutes)/60) * (-1 if direction in ['W', 'S'] else 1)
    
    latitude = city['latitude']
    longitude = city['longitude']

    lat = dms_to_deg(*re.split('[ ]', latitude))
    long = dms_to_deg(*re.split('[ ]', longitude))

    return radians(lat), radians(long)
    
def calc_d(city1, city2):
    # calculates the distance between two coordinates
    lat1, lon1 = to_degrees(city1)
    lat2, lon2 = to_degrees(city2)
    
    R = 6372.8 # km

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (sin(dlat/2)**2) + (cos(lat1) * cos(lat2) * (sin(dlon/2)**2))
    c = 2 * asin(sqrt(a)) 
    
    distance = R * c
    return distance # km

def get_distance_matrix(cities):
    # Generates a distance matrix from a list of cities with coordinates
    index_matrix = [[(i, j) for j in range(len(cities))] for i in range(len(cities))]
    distance_matrix = [[calc_d(cities[i_tup[0]], cities[i_tup[1]]) for i_tup in row] for row in index_matrix]
    return np.array([np.array(row) for row in distance_matrix])

def cal_total_distance(routine):
    num_points, = routine.shape
    return sum([distance_matrix[routine[i % num_points], routine[(i + 1) % num_points]] for i in range(num_points)])


In [271]:
i = 0
j = 7

print(cities[i])
print(cities[j])
print(distance_matrix[i][j])


{'city': ['Aberdeen, Scotland'], 'latitude': '57 9 N', 'longitude': '2 9 W'}
{'city': ['Auckland, New'], 'latitude': '36 52 S', 'longitude': '174 45 E'}
17753.055281277047


In [17]:
data = get_data()
cities = get_cities(data)
distance_matrix = get_distance_matrix(cities)

/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  del sys.path[0]
/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  app.launch_new_instance()


In [303]:
# TODO Add path back to 0

num_points = len(cities)
size_pop = 50
max_iter = 250

start = time.time()
aca = ACA_TSP(func=cal_total_distance, 
              n_dim=num_points,
              size_pop=size_pop, max_iter=max_iter,
              distance_matrix=distance_matrix)

best_x, best_y = aca.run()
stop = time.time()

time_elapsed = stop-start

result = {'routine': best_x, 'distance': best_y, 'time', time_elapsed, 'pop_size': size_pop, 'max_iter': max_iter, 'string': f'Routine: {best_x}\n\nDistance: {round(best_y)}km\n\nTime: {round(time_elapsed)} sec\n\nPopulation size: {size_pop}\n\nMax iterations: {max_iter}'}


In [317]:
print(f'Routine: {best_x}\n\nDistance: {round(best_y)}km\n\nTime: {round(time_elapsed)} sec\n\nPopulation size: {size_pop}\n\nMax iterations: {max_iter}')


Routine: [  0  41  44  12  39  64  68  61  85  21  93  65  22   3  19  47  14  34
  88 107  50 102  77 117  95 115  24  13 106  23  86   4   6  26  72  38
  81 109  40  56  29  58  36 100  98 103   5  35 101  25  76  53  60  62
  46  16  89  57  49  45 114  74  71  32  30  43  90  31  11  18  67  63
   2   9  75 119  42  78  66  70  91  99  83 113 112 110  17  84  27  96
   8  59 105  55  92   1  73  51 118   7 108  20  94  37  69  52  28  33
  82 104  79  87  80 111 116  10  54  48  97  15]

Distance: 124260km

Time: 72 sec

Population size: 50

Max iterations: 250


# TSP with genetic algorithm and multiprocessing

In [18]:
from sko.GA import GA_TSP
import multiprocessing as mp

num_points = len(cities)
size_pop = 50
max_iter = 250


ga_tsp = GA_TSP(func=cal_total_distance, n_dim=num_points, size_pop=size_pop, max_iter=max_iter, prob_mut=0.7)
best_points, best_distance = ga_tsp.run()

print(best_points, best_distance)

[ 50  83  15   0  85  34  24  58  36  81  26  19   2  88  70 112  21 113
 115  78  39  64  77  17  56 109  40  90  11   5  76  25  35 108  55 105
  59 110 111  87 116  96  27  84  82 104  79   8  52  10  28  80  54  33
  69  73   1  20  37  51 118   7 103  53 101  16  30  74  49  32  44  68
  93 117  91  99   3  23  22 102 107  48  65  47  75  67  12 106  72  38
  13  18  42  63  98  43  45  57  31   9  66  89 100  62  60  46 114  71
  94  92  29   4  86  95 119  14  41  61  97   6] [332907.91464353]


In [106]:
import multiprocessing
import time
import itertools


def GA(setting):
    max_iter = setting[0]
    prob_mut = setting[1]
    size_pop = setting[2]
    
    ga_tsp = GA_TSP(func=cal_total_distance, n_dim=120, size_pop=size_pop, max_iter=max_iter, prob_mut=prob_mut)
    best_points, best_distance = ga_tsp.run()
    return best_points, best_distance

def run(setting):
    max_iter = setting[0]
    prob_mut = setting[1]
    pop_size = setting[2]
    
    setting.append(pop_size)
    
    n_cores = multiprocessing.cpu_count()
    
    start_mp = time.time()
    
    pool = multiprocessing.Pool(processes = n_cores)
    ncore_settings = [setting for i in range(n_cores)]
    result = pool.map(rGA, ncore_settings)
    
    end_mp = time.time()

    start_seq = time.time()

    for i in range(n_cores):
        best_x, best_y = run_GA(setting)

    end_seq = time.time()

    mp_time = end_mp-start_mp
    seq_time = end_seq-start_seq
    
    print(f'\nRoutine: {list(best_x)}\n\nDistance: {round(int(best_y))}km\n\nPopulation size: {pop_size}\n\nMax iterations: {max_iter} \n\nMutation probability: {prob_mut}\n')
    print(f'With multiprocessing ({n_cores} cores) finished in {round(mp_time, 2)} seconds\n')
    print(f'Without multiprocessing finished in {round(seq_time, 2)} seconds\n')
    print(f'Running the algorithms sequantially took about {round(seq_time/mp_time, 1)} times longer\n')
    print('-'*30)
    

In [104]:
all_settings = list(itertools.product([100, 500, 1000], [0.001, 0.01, 0.05]))
all_settings = list(map(list, all_settings))
all_settings = [setting + [40] for setting in all_settings]

for setting in all_settings:
    run(setting)


Routine: [32, 109, 92, 52, 47, 14, 18, 79, 69, 28, 57, 118, 19, 58, 66, 61, 48, 82, 13, 68, 15, 83, 91, 45, 25, 67, 112, 0, 99, 42, 84, 111, 37, 94, 73, 20, 43, 85, 95, 36, 101, 60, 7, 115, 4, 65, 96, 55, 108, 26, 117, 31, 30, 98, 77, 86, 39, 88, 97, 107, 56, 51, 50, 70, 71, 59, 40, 103, 21, 93, 22, 80, 10, 24, 11, 62, 41, 72, 27, 104, 44, 64, 12, 5, 100, 74, 35, 9, 110, 29, 119, 113, 54, 1, 8, 33, 75, 87, 105, 38, 53, 89, 46, 102, 2, 90, 63, 3, 17, 81, 6, 106, 78, 23, 34, 116, 114, 49, 76, 16]

Distance: 706485km

Population size: 40

Max iterations: 100 

Mutation probability: 0.001

With multiprocessing finished in 1.3 seconds

Without multiprocessing finished in 5.57 seconds

Running the algorithms sequantially took about 4.3 times longer

------------------------------

Routine: [11, 43, 93, 97, 9, 95, 75, 91, 78, 23, 80, 116, 69, 105, 26, 72, 117, 98, 100, 114, 35, 65, 42, 40, 89, 92, 36, 60, 64, 6, 4, 29, 12, 106, 17, 54, 10, 103, 101, 2, 66, 70, 77, 71, 62, 63, 30, 37, 59, 108